# 实验五：投资组合优化分析 —— 协方差矩阵的计算

## 数据处理

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('../../实验五/assets/data/3-B_EXAM4.csv', encoding='gbk')
data

,日期,市场指数,组合1,组合2,组合3,组合4,组合5,无风险利率,市场溢酬因子,市值因子,账面市值比因子
0,20090109,1904.86,1727.01,1281.30,1591.43,2469.13,1822.58,0.000043,0.0204,0.0083,0.0010
1,20090112,1900.35,1715.86,1300.27,1590.85,2488.00,1829.90,0.000042,0.0044,0.0102,-0.0021
2,20090113,1863.37,1667.97,1273.80,1551.21,2407.01,1774.56,0.000041,-0.0249,-0.0019,0.0074
3,20090114,1928.87,1750.08,1329.63,1607.21,2463.77,1817.24,0.000041,0.0427,0.0029,-0.0022
4,20090115,1920.21,1739.64,1334.24,1614.49,2456.65,1814.10,0.000040,0.0033,0.0079,-0.0031
...,...,...,...,...,...,...,...,...,...,...,...
2422,20181224,2527.01,1289.12,1634.66,1869.08,7276.91,5275.89,0.000088,0.0056,-0.0008,-0.0045
2423,20181225,2504.82,1259.94,1605.20,1846.48,7256.91,5251.07,0.000090,-0.0088,-0.0034,-0.0057
2424,20181226,2498.29,1250.42,1598.94,1843.40,7216.06,5193.21,0.000091,-0.0030,0.0048,0.0030
2425,20181227,2483.09,1236.36,1587.71,1831.41,7233.59,5144.72,0.000091,-0.0080,-0.0095,0.0024


In [2]:
# 提取各组合价格数据和其他相关变量
p1 = data.iloc[:, 2].values  # 组合1价格
p2 = data.iloc[:, 3].values  # 组合2价格
p3 = data.iloc[:, 4].values  # 组合3价格
p4 = data.iloc[:, 5].values  # 组合4价格
p5 = data.iloc[:, 6].values  # 组合5价格
rf = data.iloc[:, 7].values  # 无风险利率
mkt = data.iloc[:, 8].values  # 市场溢酬因子

In [3]:
# 计算各组合的对数收益率
# 对数收益率公式: r_t = ln(P_t) - ln(P_{t-1})
r1 = np.log(p1[1:]) - np.log(p1[:-1])
r2 = np.log(p2[1:]) - np.log(p2[:-1])
r3 = np.log(p3[1:]) - np.log(p3[:-1])
r4 = np.log(p4[1:]) - np.log(p4[:-1])
r5 = np.log(p5[1:]) - np.log(p5[:-1])

In [4]:
# 计算超额收益率（对数收益率减去无风险利率）
rexc1 = r1 - rf[1:]  # 组合1超额收益率
rexc2 = r2 - rf[1:]  # 组合2超额收益率
rexc3 = r3 - rf[1:]  # 组合3超额收益率
rexc4 = r4 - rf[1:]  # 组合4超额收益率
rexc5 = r5 - rf[1:]  # 组合5超额收益率

In [5]:
# 合并所有组合的超额收益率为一个矩阵
R = np.concatenate([rexc1[:, None],  # 变为列向量，即通过[:, None]添加一个新的轴（从形状(n,)变为(n,1)）
                    rexc2[:, None],
                    rexc3[:, None], 
                    rexc4[:, None],
                    rexc5[:, None]], 
                   axis=1)

R

array([[-0.00651918,  0.01465475, -0.00040652,  0.00757131,  0.00396624],
       [-0.02834809, -0.02060838, -0.02527419, -0.03313486, -0.03074982],
       [ 0.04801318,  0.04285515,  0.03542348,  0.02326638,  0.02372536],
       ...,
       [-0.00767561, -0.00399845, -0.00176043, -0.00573602, -0.01117086],
       [-0.01139892, -0.00713918, -0.00661653,  0.00233536, -0.00947206],
       [-0.00874331,  0.00274454,  0.00902862,  0.01574632,  0.00634242]])

## 方差矩阵计算

In [6]:
# 计算样本协方差矩阵
# Cov(i,j)=E[(ri​−μi​)(rj​−μj​)]
# rowvar=False 参数指定数据是按列组织的，即每一列是一个变量（一个投资组合的收益率）
Cov_Sample = np.mat(np.cov(R, rowvar=False))

Cov_Sample

matrix([[0.00036161, 0.00031648, 0.00026988, 0.00021371, 0.00019017],
        [0.00031648, 0.00036965, 0.00028689, 0.00024158, 0.00022704],
        [0.00026988, 0.00028689, 0.00029853, 0.00021888, 0.00020663],
        [0.00021371, 0.00024158, 0.00021888, 0.00026741, 0.0002325 ],
        [0.00019017, 0.00022704, 0.00020663, 0.0002325 , 0.0002925 ]])

In [7]:
# 使用单因子模型（CAPM）估计因子模型协方差矩阵
# Ri​−Rf​=αi​+βi​(Rm​−Rf​)+εi​

# 矩阵X，包含截距项和市场因子收益率
'''
创建一个包含两列的矩阵

第一列是全1的截距项(对应α)

第二列是市场超额收益率(对应β系数)
'''
X = np.mat(np.concatenate(
        [
            np.ones((len(mkt)-1, 1)), # 创建一个形状为(len(mkt)-1, 1)的全1数组，这将作为线性回归中的截距项（常数项），len(mkt)-1表示观测数量，与之前计算的收益率数组长度相匹配
            mkt[1:, None]  # mkt[1:]选取市场溢酬因子从第二个观测值开始的所有值；, None相当于[:, np.newaxis]，将一维数组转换为列向量
        ], 
        axis=1
    )
)
Y = np.mat(R)  # 各组合超额收益率

# 使用最小二乘法估计α和β系数
AB_hat = np.dot(
    np.linalg.inv(np.dot(np.transpose(X), X)), np.dot(np.transpose(X), Y)
)
ALPHA = AB_hat[0]  # α值（非系统性收益）
BETA = AB_hat[1]   # β值（系统性风险系数）

In [8]:
# 计算残差（非系统性风险）
# 真实 - 预测
RESD = Y - X*AB_hat

# 计算市场因子的方差 σm^2​
covfactor = np.cov(mkt[1:])

# 计算残差协方差矩阵（假设残差之间相互独立，只保留对角线元素）
# np.diag(np.diag(...)) 只保留对角线元素，其他元素设为0
# 得到的 covresidual 是一个对角矩阵，对角线元素代表各资产特质风险的方差，记作矩阵 D
covresidual = np.diag(np.diag(np.cov(RESD, rowvar=False)))

# 根据单因子模型计算协方差矩阵
# ΣFactor​=ββ′σm^2​+D
Cov_Factor = BETA.T*covfactor*BETA + covresidual

Cov_Factor

matrix([[0.00036161, 0.00028866, 0.00026249, 0.00022798, 0.00022049],
        [0.00028866, 0.00036965, 0.00027923, 0.00024251, 0.00023454],
        [0.00026249, 0.00027923, 0.00029853, 0.00022053, 0.00021329],
        [0.00022798, 0.00024251, 0.00022053, 0.00026741, 0.00018524],
        [0.00022049, 0.00023454, 0.00021329, 0.00018524, 0.0002925 ]])

In [9]:
# 计算收缩估计协方差矩阵（Shrinkage Covariance Matrix）
c = 0.5  # 收缩强度参数

# ΣShrink​=cΣSample​+(1−c)ΣFactor​
Cov_Shrink = c*Cov_Sample + (1-c)*Cov_Factor

Cov_Shrink

matrix([[0.00036161, 0.00030257, 0.00026619, 0.00022084, 0.00020533],
        [0.00030257, 0.00036965, 0.00028306, 0.00024205, 0.00023079],
        [0.00026619, 0.00028306, 0.00029853, 0.0002197 , 0.00020996],
        [0.00022084, 0.00024205, 0.0002197 , 0.00026741, 0.00020887],
        [0.00020533, 0.00023079, 0.00020996, 0.00020887, 0.0002925 ]])

最优投资组合权重计算

均值-方差投资组合优化问题可以表述为：

最小化：$w'\Sigma w$ (投资组合方差)

约束条件：
- $w'\mu = \mu_p$ (目标期望收益率)
- $w'1 = 1$ (权重和为1)

其中：
- $w$ 是权重向量
- $\Sigma$ 是协方差矩阵
- $\mu$ 是期望收益率向量
- $\mu_p$ 是目标期望收益率
- $1$ 是全1向量

使用拉格朗日乘子法，最优权重的解为：

$$w^* = \Sigma^{-1}A'(A\Sigma^{-1}A')^{-1}b$$

其中：
- $A$ 是约束条件矩阵 $[\mu', 1']'$
- $b$ 是约束条件向量 $[\mu_p, 1]'$

代码分别使用收缩估计协方差矩阵和样本协方差矩阵计算最优权重，以便比较两种方法的结果。

In [10]:
# 计算各组合平均超额收益率
# 对矩阵R按列计算平均值，得到每个投资组合的平均超额收益率
uhat = np.mean(R, axis=0)  # 形状为(5,)的数组，如[0.002, 0.003, 0.0025, 0.0018, 0.0022]

# 构建约束条件矩阵A（要求期望收益率为up，权重和为1）
# A矩阵将用于表示投资组合优化问题的两个约束条件：
# 1. 组合收益率 = 目标收益率 (w'μ = μ_p)
# 2. 权重之和 = 1 (w'1 = 1)
A = np.mat(np.concatenate([uhat[:, None], np.ones((len(uhat), 1))], axis=1)).T
# 详细步骤：
# 1. uhat[:, None]：将平均收益率向量转为列向量，如[[0.002], [0.003], [0.0025], [0.0018], [0.0022]]
# 2. np.ones((len(uhat), 1))：创建全1列向量，如[[1], [1], [1], [1], [1]]
# 3. np.concatenate(..., axis=1)：水平拼接，得到[[0.002, 1], [0.003, 1], ...]
# 4. .T：转置，得到最终A矩阵:
#    [[0.002, 0.003, 0.0025, 0.0018, 0.0022],  # 第一行：各资产平均收益率
#     [1,     1,     1,      1,      1     ]]  # 第二行：权重和为1的约束

# 计算所有组合的平均超额收益率
# 这将作为目标收益率，即我们希望最优组合的期望收益率等于所有组合的平均收益率
up = np.mean(uhat)  # 如果uhat=[0.002, 0.003, 0.0025, 0.0018, 0.0022]，则up=0.0023

# 构建约束条件向量b
# b向量代表约束条件的右侧常数：
# 1. 第一个元素：目标收益率up
# 2. 第二个元素：权重和为1
b = np.mat(np.array([up, 1])[:, None])  # 结果为矩阵形式的列向量[[0.0023], [1]]

# 使用收缩估计协方差矩阵求解最优权重
# 这是均值-方差优化问题的解析解公式：ω = Σ^(-1)A'(AΣ^(-1)A')^(-1)b
# 数学背景：我们要最小化 w'Σw (投资组合方差)，同时满足 Aw = b (约束条件)
# 使用拉格朗日乘子法求解这个约束优化问题
omega_Shrink = Cov_Shrink.I * A.T * (A * Cov_Shrink.I * A.T).I * b
# 步骤分解：
# 1. Cov_Shrink.I：收缩估计协方差矩阵的逆
# 2. A.T：约束条件矩阵的转置
# 3. (A * Cov_Shrink.I * A.T).I：中间矩阵的逆(2×2矩阵)
# 4. 最终结果是最优权重向量，形状为(5,1)
print(omega_Shrink)  # 打印使用收缩估计协方差矩阵得到的最优权重

# 使用样本协方差矩阵求解最优权重（用于比较）
# 公式相同，但使用样本协方差矩阵代替收缩估计协方差矩阵
omega_Sample = Cov_Sample.I * A.T * (A * Cov_Sample.I * A.T).I * b
# 这里我们计算两种不同协方差矩阵估计方法得到的最优权重，以便比较结果差异
# 理论上，收缩估计方法得到的权重应该更稳健，因为它减少了样本估计的极端值
print()
print(omega_Sample)  # 打印使用样本协方差矩阵得到的最优权重

[[ 0.32532288]
 [-0.21248216]
 [ 0.38898507]
 [ 0.23881412]
 [ 0.25936009]]

[[ 0.39904587]
 [-0.32861957]
 [ 0.38202395]
 [ 0.23869463]
 [ 0.30885513]]


结果分析

最终输出的 `omega_Shrink` 和 `omega_Sample` 是两种不同协方差矩阵估计方法下的最优投资组合权重。比较两种结果可以看出，使用不同的协方差矩阵估计方法会导致不同的最优权重分配。

收缩估计方法通常能提供更为稳健的结果，因为它结合了样本协方差矩阵的信息和因子模型的先验结构，可以减少极端估计值的影响。从结果可以看出，两种方法得到的权重有明显差异，特别是对组合1和组合2的权重分配。